In [1]:
import os
# NOTEBOOK_NAME = "ex6-finetune-add-desc-text-ep4-8"
NOTEBOOK_NAME = "ex6-finetune-add-desc-text-ep9-10"
os.environ['WANDB_NOTEBOOK_NAME'] = NOTEBOOK_NAME

In [2]:
OUTPUT_DIR = f"/notebooks/kaggle_lecr/output/{NOTEBOOK_NAME}"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [3]:
!nvidia-smi

Mon Feb 20 06:49:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A4000    Off  | 00000000:00:05.0 Off |                  Off |
| 41%   45C    P8    13W / 140W |      0MiB / 16376MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import os
os.system('pip install python-dotenv')

from dotenv import load_dotenv
load_dotenv()

True

In [5]:
class CFG:
    debug = False
    upload_data = True
    batch_size = 32#64 #128# 256# 384 # 24GBならbs64でいける, 16GBならbs32
    n_folds = 5
    seed = 42
    num_epochs = 2#5#3#10
    wandb = True
    data_url = "/notebooks/kaggle_lecr/data/learning-equality-curriculum-recommendations/"
    uns_model = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"

In [6]:
# ====================================================
# wandb
# ====================================================
if CFG.wandb:
    os.system('pip install wandb')
    import wandb

    try:
        # for kaggle
        # from kaggle_secrets import UserSecretsClient
        # user_secrets = UserSecretsClient()
        # secret_value_0 = user_secrets.get_secret("wandb_api")
        
        # for paperspace
        secret_value_0 = os.getenv('WANDB_API_KEY')
        wandb.login(key=secret_value_0)
        
        anony = None
    except:
        anony = "must"
        print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')


    def class2dict(f):
        return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

    run = wandb.init(project='LECR',
                     entity="sinchir0",
                     name=NOTEBOOK_NAME,
                     config=class2dict(CFG),
                     group="fine-tuning",
                     job_type="train",
                     anonymous=anony)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 20.1 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=8a80c39e91e6f096ca2b7805c8fef5d0e78a1683f11a337ec093c017f6c6c14f
  Stored in directory: /root/.cache/pip/wheels/20/7c/09/4ad42725a29fce4bc21137c7f25f062b3655a4aea5b0e8d9a2
Successfully built pathtools


wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find ex6-finetune-add-desc-text-ep9-10.
wandb: Currently logged in as: sinchir0. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
os.system("pip3 install torch==1.12.0 --extra-index-url https://download.pytorch.org/whl/cu116")

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 GB 1.1 MB/s eta 0:00:00


0

In [9]:
import torch
wandb.log({"GPU":torch.cuda.get_device_name()})

In [10]:
os.system("pip install scikit-learn==1.2.1")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 68.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2


0

In [11]:
!pip -qqq install sentence-transformers
!pip -qqq install datasets
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sentence_transformers import SentenceTransformer, models, InputExample, losses
from datasets import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import StratifiedGroupKFold, KFold, StratifiedKFold, GroupKFold

In [12]:
DATA_PATH = "/kaggle/input/learning-equality-curriculum-recommendations/"
topics = pd.read_csv(CFG.data_url + "topics.csv")
content = pd.read_csv(CFG.data_url + "content.csv")
correlations = pd.read_csv(CFG.data_url + "correlations.csv")
correlations.shape

(61517, 2)

In [13]:
topics = topics.fillna("")
content = content.fillna("")

In [14]:
def add_parent_text(topics: pd.DataFrame):
    id_full_text = {}
    id_to_text = {}
    for i, row in topics.iterrows():
        id_to_text[row.id] = [row.title, row.parent]
    print('done')
    def get_full_text(id):
        if id in id_full_text:
            return id_full_text[id]
        data = id_to_text[id]
        # full_text = f'{data[0]} < {get_full_text(data[1])}' if data[1] != '' else data[0]
        full_text = f'{data[0]} > {get_full_text(data[1])}' if data[1] != '' else data[0]
        id_full_text[id] = full_text
        return full_text
    tqdm.pandas()
    topics['title'] = topics.id.progress_apply(get_full_text)
    del id_full_text
    del id_to_text
    return topics

topics = add_parent_text(topics)

done


  0%|          | 0/76972 [00:00<?, ?it/s]

In [15]:
topics.rename(columns=lambda x: "topic_" + x, inplace=True)
content.rename(columns=lambda x: "content_" + x, inplace=True)

In [16]:
correlations["content_id"] = correlations["content_ids"].str.split(" ")
corr = correlations.explode("content_id").drop(columns=["content_ids"])
corr

,topic_id,content_id
0,t_00004da3a1b2,c_1108dd0c7a5d
0,t_00004da3a1b2,c_376c5a8eb028
0,t_00004da3a1b2,c_5bc0e1e2cba0
0,t_00004da3a1b2,c_76231f9d0b5e
1,t_00068291e9a4,c_639ea2ef9c95
...,...,...
61513,t_fff9e5407d13,c_d64037a72376
61514,t_fffbe1d5d43c,c_46f852a49c08
61514,t_fffbe1d5d43c,c_6659207b25d5
61515,t_fffe14f1be1e,c_cece166bad6a


In [17]:
corr = corr.merge(topics, how="left", on="topic_id")
corr = corr.merge(content, how="left", on="content_id")
corr.head()

,topic_id,content_id,topic_title,topic_description,topic_channel,topic_category,topic_level,topic_language,topic_parent,topic_has_content,content_title,content_description,content_kind,content_text,content_language,content_copyright_holder,content_license
0,t_00004da3a1b2,c_1108dd0c7a5d,Откриването на резисторите > Открития и проект...,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True,Молив като резистор,"Моливът причинява промяна в отклонението, подо...",video,,bg,,
1,t_00004da3a1b2,c_376c5a8eb028,Откриването на резисторите > Открития и проект...,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True,Да чуем променливото съпротивление,Тук чертаем линия на лист хартия и я използвам...,video,,bg,,
2,t_00004da3a1b2,c_5bc0e1e2cba0,Откриването на резисторите > Открития и проект...,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True,Променлив резистор (реостат) с графит от молив,Използваме сърцевината на молива (неговия граф...,video,,bg,,
3,t_00004da3a1b2,c_76231f9d0b5e,Откриването на резисторите > Открития и проект...,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True,Последователно свързване на галваничен елемент...,"Защо отклонението се променя, когато се свърже...",video,,bg,,
4,t_00068291e9a4,c_639ea2ef9c95,Entradas e saídas de uma função > Álgebra: fun...,Entenda um pouco mais sobre funções.,8e286a,source,4,pt,t_d14b6c2a2b70,True,Dados e resultados de funções: gráficos,Encontre todas as entradas que correspondem a ...,exercise,,pt,,


In [18]:
corr["topic_use_text"] = corr["topic_title"] + " " + corr["topic_description"]
# corr["content_use_text"] = corr["content_title"] + " " + corr["content_description"] + " " + corr["content_text"]
corr["content_use_text"] = corr["content_title"] + " " + corr["content_description"]

In [19]:
corr.head()

,topic_id,content_id,topic_title,topic_description,topic_channel,topic_category,topic_level,topic_language,topic_parent,topic_has_content,content_title,content_description,content_kind,content_text,content_language,content_copyright_holder,content_license,topic_use_text,content_use_text
0,t_00004da3a1b2,c_1108dd0c7a5d,Откриването на резисторите > Открития и проект...,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True,Молив като резистор,"Моливът причинява промяна в отклонението, подо...",video,,bg,,,Откриването на резисторите > Открития и проект...,Молив като резистор Моливът причинява промяна ...
1,t_00004da3a1b2,c_376c5a8eb028,Откриването на резисторите > Открития и проект...,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True,Да чуем променливото съпротивление,Тук чертаем линия на лист хартия и я използвам...,video,,bg,,,Откриването на резисторите > Открития и проект...,Да чуем променливото съпротивление Тук чертаем...
2,t_00004da3a1b2,c_5bc0e1e2cba0,Откриването на резисторите > Открития и проект...,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True,Променлив резистор (реостат) с графит от молив,Използваме сърцевината на молива (неговия граф...,video,,bg,,,Откриването на резисторите > Открития и проект...,Променлив резистор (реостат) с графит от молив...
3,t_00004da3a1b2,c_76231f9d0b5e,Откриването на резисторите > Открития и проект...,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True,Последователно свързване на галваничен елемент...,"Защо отклонението се променя, когато се свърже...",video,,bg,,,Откриването на резисторите > Открития и проект...,Последователно свързване на галваничен елемент...
4,t_00068291e9a4,c_639ea2ef9c95,Entradas e saídas de uma função > Álgebra: fun...,Entenda um pouco mais sobre funções.,8e286a,source,4,pt,t_d14b6c2a2b70,True,Dados e resultados de funções: gráficos,Encontre todas as entradas que correspondem a ...,exercise,,pt,,,Entradas e saídas de uma função > Álgebra: fun...,Dados e resultados de funções: gráficos Encont...


In [20]:
corr["topic_use_text"].apply(lambda x: len(x.split())).max()

880

In [21]:
corr["content_use_text"].apply(lambda x: len(x.split())).max()

1573

In [22]:
corr["set"] = corr[["topic_use_text", "content_use_text"]].values.tolist()
train_df = pd.DataFrame(corr["set"])

In [23]:
train_df["set"][0]

['Откриването на резисторите > Открития и проекти > Физика > Наука > Khan Academy (български език) Изследване на материали, които предизвикват намаление в отклонението, когато се свържат последователно с нашия измервателен уред. ',
 'Молив като резистор Моливът причинява промяна в отклонението, подобно на ефекта при свързването на\nелектромагнит. Защо?\n\n']

In [24]:
if CFG.debug:
    train_df = train_df[:1000]

In [25]:
dataset = Dataset.from_pandas(train_df)

In [26]:
dataset

Dataset({
    features: ['set', '__index_level_0__'],
    num_rows: 279919
})

In [27]:
train_examples = []
train_data = dataset["set"]
n_examples = dataset.num_rows

for i in range(n_examples):
    example = train_data[i]
    if example[0] == None: #remove None
        print(example)
        continue        
    train_examples.append(InputExample(texts=[str(example[0]), str(example[1])]))

In [28]:
# model = SentenceTransformer(CFG.uns_model)
# model = SentenceTransformer("/notebooks/kaggle_lecr/output/ex6-finetune-add-desc-text-ep0-3")
model = SentenceTransformer("/notebooks/kaggle_lecr/output/ex6-finetune-add-desc-text-ep4-8")

In [29]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=CFG.batch_size)
train_loss = losses.MultipleNegativesRankingLoss(model=model)
if CFG.debug:
    CFG.num_epochs = 1
warmup_steps = int(len(train_dataloader) * CFG.num_epochs * 0.1) #10% of train data

In [30]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=CFG.num_epochs,
          save_best_model = True,
          output_path=OUTPUT_DIR,
          warmup_steps=warmup_steps)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/8748 [00:00<?, ?it/s]

Iteration:   0%|          | 0/8748 [00:00<?, ?it/s]

# Upload

In [31]:
import os
os.system("pip install kaggle")
os.system("mkdir -p ~/.kaggle/")
os.system("cp /notebooks/kaggle_lecr/kaggle.json ~/.kaggle/")
os.system("chmod 600 ~/.kaggle/kaggle.json")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 22.2 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73049 sha256=6227fbc4cca6b97abf58576a083433302517d083769c5b5f906d6efa47c25dc9
  Stored in directory: /root/.cache/pip/wheels/50/0a/6a/77a4f3a534f0e5fd0909a376bbdfc88238a43eb2ac35947dc7
Successfully built kaggle


0

In [32]:
from kaggle.api.kaggle_api_extended import KaggleApi
import json

def dataset_create_new(dataset_name: str, upload_dir: str):
    if "_" in dataset_name:
        raise ValueError("datasetの名称に_の使用は禁止です")
    dataset_metadata = {}
    dataset_metadata['id'] = f'sinchir0/{dataset_name}'
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = dataset_name
    with open(os.path.join(upload_dir, 'dataset-metadata.json'), 'w') as f:
        json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    api.dataset_create_new(folder=upload_dir, convert_to_csv=False, dir_mode='tar')

if CFG.upload_data:
    print(f"Create Dataset name:{NOTEBOOK_NAME}, output_dir:{OUTPUT_DIR}")
    dataset_create_new(dataset_name=NOTEBOOK_NAME, upload_dir=OUTPUT_DIR)

Create Dataset name:ex6-finetune-add-desc-text-ep9-10, output_dir:/notebooks/kaggle_lecr/output/ex6-finetune-add-desc-text-ep9-10
Starting upload for file pytorch_model.bin


100%|██████████| 1.04G/1.04G [00:13<00:00, 79.8MB/s]


Upload successful: pytorch_model.bin (1GB)
Starting upload for file sentencepiece.bpe.model


100%|██████████| 4.83M/4.83M [00:00<00:00, 7.55MB/s]


Upload successful: sentencepiece.bpe.model (5MB)
Starting upload for file README.md


...

Upload successful: sentence_bert_config.json (53B)
Starting upload for file special_tokens_map.json


100%|██████████| 280/280 [00:00<00:00, 491B/s]


Upload successful: special_tokens_map.json (280B)
Starting upload for file eval.tar


100%|██████████| 10.0k/10.0k [00:00<00:00, 16.6kB/s]


Upload successful: eval.tar (10KB)


In [33]:
wandb.finish()

GPU,NVIDIA RTX A4000
